In [1]:
import tensorflow as tf
import numpy as np

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


tf.name_scope
 - 블록으로, 텐서보드에서 한 계층의 내부를 표현

name argument
 - 모든 텐서에 name을 기입할 수 있음 (연산, 활성화함수, 홀더, 변수 모두)

In [2]:
data = np.loadtxt('./DataSet/data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

global_step = tf.Variable(0, trainable=False, name='global_step')

In [3]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1, 1), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1, 1), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('Output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1, 1), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
train = optimizer.minimize(cost, global_step=global_step)

tf.summary.scalar
 - 손실값을 추적하기 위해 수집할 값을 지정하는 코드를 작성
 - scalar의 경우는 값이 하나인 텐서를 수집할 때 사용

tf.summary
 - histogram, image, audio 등 다양한 값을 수집하는 함수를 기본 제공

In [11]:
tf.summary.scalar('cost', cost)

<tf.Tensor 'cost_1:0' shape=() dtype=string>

merge_all
 - 함수로 앞에 지정한 텐서들을 수집
 
FileWriter
 - 그래프와 텐서들의 값을 저장할 디렉터리 설정

In [16]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./DataSet/modelTensorboard2')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restores(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
merged= tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [17]:
for step in range(100):
    sess.run(train, feed_dict={X: x_data, Y: y_data})
    if step // 10 == 0:
        print(sess.run(global_step), ':', sess.run(cost, feed_dict={X: x_data, Y: y_data}))

1 : 0.8441933
2 : 0.65183145
3 : 0.56805223
4 : 0.55365664
5 : 0.55266494
6 : 0.5549874
7 : 0.55677825
8 : 0.55466825
9 : 0.55163497
10 : 0.5501375


In [18]:
summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

In [19]:
saver.save(sess, './DataSet/model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
actual = tf.argmax(Y, 1)

print('predction : ', sess.run(prediction, feed_dict={X: x_data}))
print('Actual    :', sess.run(actual, feed_dict={Y: y_data}))

is_correct = tf.reduce_mean(tf.cast(tf.equal(prediction, actual), tf.float32))
print('is_correct:', sess.run(is_correct, feed_dict={X: x_data, Y: y_data}))

predction :  [0 1 2 0 0 2]
Actual    : [0 1 2 0 0 2]
is_correct: 1.0
